# Chapter 4: Object Metaphor

# Representing Bank Account
Suppose that we wish develop a program to keep track of bank account. The data required are name, and amount. The methods can be applied are withdraw, and deposit.

## Using `Tuple`
Naively, we may try to use `tuple`

In [106]:
def make_new_acc(name, initial_amount):
    return name, initial_amount

def account_name(acc):
    return acc[0]

def account_amount(acc):
    return acc[1]

def withdraw(acc, amt):
    if account_amount(acc) < amt:
        print("Insufficient Fund") 
        return acc
    return make_new_acc(account_name(acc), 
                        account_amount(acc) - amt)

def deposit(acc, amt):
    if amt <= 0:
        print("Deposit amount must be positive")
        return acc
    return make_new_acc(account_name(acc), 
                        account_amount(acc) + amt) 

In [107]:
acc = make_new_acc("Teng Man", 100) 
print(acc, type(acc))

('Teng Man', 100) <class 'tuple'>


In [108]:
acc = withdraw(acc, 70)
acc

('Teng Man', 30)

In [109]:
acc = deposit(acc, 30)
acc

('Teng Man', 60)

In [110]:
deposit(acc, 30)
deposit(acc, 30)
deposit(acc, 30)
acc

('Teng Man', 60)

## Using `dict`

We may also use the Python `dict` to represent the bank account

In [111]:
def call_dict_method(obj, method, *args):
    return obj[method](obj, *args)

def make_new_acc(name, initial_amount):
    def withdraw(acc, amt):
        if acc["amount"] < amt:
            return "insufficient fund"
        acc["amount"]  -= amt
        return acc["amount"]
    
    def deposit(acc, amt):
        if amt <= 0:
            return "deposit amount must be positive"
        acc["amount"] += amt
        return acc["amount"]
    
    def display(acc):
        print(f'''Bank_Account('{acc["name"]}', {acc["amount"]})''')
    
    return {
        "name":name,
        "amount": initial_amount,
        "withdraw": withdraw,
        "deposit": deposit,
        "display": display
    }

In [112]:
teng_man_acc = make_new_acc("Teng Man", 100)
call_dict_method(teng_man_acc, "display")

Bank_Account('Teng Man', 100)


In [113]:
call_dict_method(teng_man_acc, "withdraw", 20)

80

In [114]:
call_dict_method(teng_man_acc, "display")

Bank_Account('Teng Man', 80)


In [115]:
call_dict_method(teng_man_acc, "deposit", 120)

200

In [116]:
call_dict_method(teng_man_acc, "display")

Bank_Account('Teng Man', 200)


## Lexical Closure
Or we may exploit `nonlocal`, lexical scoping

In [117]:
def make_new_acc(name, amount):
    def withdraw(amt):
        nonlocal amount
        if amount < amt:
            return "insufficient fund"
        amount -= amt
        return amount
    
    def deposit(amt):
        nonlocal amount
        if amt <= 0:
            return "deposit amount must be positive"
        amount += amt
        return amount
    
    def display():
        print(f'''Bank_Account('{name}', {amount})''')
        
    def dispatch(msg):
        if msg == "withdraw":
            return withdraw
        elif msg == "deposit":
            return deposit
        elif msg == "display":
            return display
        else:
            raise Exception(f"Unknown Message to the object Bank Account {msg}")
    return dispatch

In [118]:
acc = make_new_acc("Hakurei", 100)
acc("display")()

Bank_Account('Hakurei', 100)


In [119]:
acc("withdraw")(20)
acc("display")()

Bank_Account('Hakurei', 80)


In [120]:
acc("withdraw")(20)
acc("display")()

Bank_Account('Hakurei', 60)


In [121]:
acc("deposit")(60)
acc("display")()

Bank_Account('Hakurei', 120)


The last 2 use the message passing technique where they dispatch on message and return function. And the functions are encapsulated within the function. 

Many mainstream programming languages also capture such pattern and abstract these as **object-oriented programming**. They also usually add more feature such as inheritance, data abstraction, and control access.

Usually, it is encouraged to use Programming Language features. Indeed, we will use `class` for convenience in data abstraction and program later. Previously proposed programs are just to show that even without the Programming Languages explcitly support OOP, we can still do something similar. Indeed, Python OOP uses something similiar to `dict` to represent object. Lua is lack of `class` construct, but it can use its `table` data structure to emulate OOP.

## Using `class`  (recomended)

In [122]:
class Bank_Account:
    # __init__ is a magic method of Python, it is called every time constructed
    def __init__(self, name, amount):
        self.name = name
        self.amount = amount
    # first parameter of method inside a class, always refer to the object
    def withdraw(self, amt):
        if self.amount < amt:
            return "insufficient fund"
        self.amount -= amt
    
    def deposit(self, amt):
        if amt <= 0:
            return "deposit amount must be positive"
        self.amount += amt
        return self.amount
    # another magic method that it is invoked when REPL ask for its output
    # the return type is expected to string type
    def __repr__(self):
        return f'''Bank_Account('{self.name}', {self.amount})'''

In [123]:
marisa_acc = Bank_Account("marisa", 200) # create object of Bank_Account; __init__ is invoked
marisa_acc

Bank_Account('marisa', 200)

To access the member of a class, we may the syntax as below:

```
<obj_name>.<member_name>
```

In [124]:
marisa_acc.amount

200

In [125]:
marisa_acc.withdraw

<bound method Bank_Account.withdraw of Bank_Account('marisa', 200)>

In [126]:
marisa_acc.withdraw(20) # it is same as calling Bank_Account.withdraw(marisa_acc, 20)
marisa_acc

Bank_Account('marisa', 180)

# Problem of Object: Sameness and Change

Suppose that we have two bank account object.

In [127]:
frisk_acc1 = Bank_Account('frisk', 100)
frisk_acc2 = Bank_Account('frisk', 100)
print(frisk_acc1)
print(frisk_acc2)

Bank_Account('frisk', 100)
Bank_Account('frisk', 100)


The question is that are `frisk_acc1` and `frisk_acc2` the same thing?

No, they are not the same thing in the sense that they don't share the same effect, although they created by the same values.

In [128]:
frisk_acc1.withdraw(30)
frisk_acc1.withdraw(25)
frisk_acc1

Bank_Account('frisk', 45)

In [129]:
frisk_acc2.withdraw(25)
frisk_acc2

Bank_Account('frisk', 75)

Therefore, Python provide another keyword `is` is to test whether two object are the same rather than equality.

In [130]:
print(frisk_acc1 is frisk_acc1)
print(frisk_acc2 is frisk_acc1)

True
False


To correctly reason about the programs involving objects, we must somehow know their **history** and **order** of which expressions are evaluated.

There is another problem of object.

In [131]:
x = 2
y = 3
y += 8
print(x,y)

2 11


However, this is not true for object

In [132]:
sakuya_acc1 = Bank_Account('sakuya', 100)
sakuya_acc2 = sakuya_acc1

In [133]:
sakuya_acc1.withdraw(20)

In [134]:
sakuya_acc2

Bank_Account('sakuya', 80)

This is because the expression `sakuya_acc2 = sakuya_acc1` that `sakuya_acc2` is merely aliasing instead assigned a new value. In other word, `sakuya_acc1` and `sakuya_acc2` point to the same object. It can be expected that the expression below is evaluated to be true.

In [135]:
sakuya_acc1 is sakuya_acc2

True

Interestingly, the bank account representation that use `tuple` doesn't share this defect as long as you don't reassign. Hence, the tuple representation is not object in this sense. Indeed, `tuple` is immutable.

However, `list` share the same defect because `list` is object too.

In [136]:
xs = list(range(3))
ys = [xs,xs,xs]
print(xs)
print(ys)

[0, 1, 2]
[[0, 1, 2], [0, 1, 2], [0, 1, 2]]


In [137]:
xs.extend(["bug", "never gonna give you up"])
ys

[[0, 1, 2, 'bug', 'never gonna give you up'],
 [0, 1, 2, 'bug', 'never gonna give you up'],
 [0, 1, 2, 'bug', 'never gonna give you up']]

Data structures with these ability is called **mutable**. Lack of this ability include `string` and `tuple`, they are called **immutable**.

Python default arguments are evaluated once, hence if the default argument is mutable, then it can change over time causing confuing bugs.

Tips: default arguments should be immutable data

In [138]:
DEFAULT_CURRENCY = [1,2,5,10,20,50,100,200]
def count_change(amount, coins=DEFAULT_CURRENCY):
    if amount == 0:
        return 1
    elif amount < 0:
        return 0
    elif coins == []:
        return 0
    else:
        return count_change(amount, coins[1:]) + count_change(amount - coins[0], coins)
count_change(100)

4563

In [139]:
DEFAULT_CURRENCY.pop(0)
count_change(100) 

197

# Exercise

1. `tuple` and `string` are immutable. For example,

In [140]:
t = tuple(range(3))
t[0] = t[0] + 1

TypeError: 'tuple' object does not support item assignment

In [142]:
hakurei = "hakurei"
hakurei[0] = "H"

TypeError: 'str' object does not support item assignment

This is known as **immutablity**. Since `tuple` is immutable, fix the `count_change` in previously shown.

In [143]:
DEFAULT_CURRENCY = [1,2,5,10,20,50,100,200]
def count_change(amount, coins=DEFAULT_CURRENCY):
    if amount == 0:
        return 1
    elif amount < 0:
        return 0
    elif coins == []:
        return 0
    else:
        return count_change(amount, coins[1:]) + count_change(amount - coins[0], coins)
count_change(100)

4563

2. We expected that arguments are usually local to the function for example

In [144]:
def capitalize_words(msg):
    msg = msg.split()
    msg = map(lambda s:s.capitalize(), msg)
    msg = " ".join(msg)
    return msg
msg = "We expected that arguments are usually local to the function for example"
print(capitalize_words(msg))
print(msg)

We Expected That Arguments Are Usually Local To The Function For Example
We expected that arguments are usually local to the function for example


In [145]:
def sqr_list(li):
    for i, itm in enumerate(li):
        li[i] = li[i]*li[i]
    return li
xs = list(range(1,4))
ys = sqr_list(xs)
xs

[1, 4, 9]

However, this is not true to mutable data as in the example above. We might expect that argument `xs` should not change but it changes. Try to come out similiar bug using `dict`.

3. Consider a game object which has name, health, decrease health if hit, increase healht if heal and die if reach zero health. Complete the code below, and demonstrate how to use it.

In [146]:
class NPC:
    def __init__(self):
        pass
    def heal(self):
        pass
    def hit(self):
        pass
    def is_dead(self):
        pass